In [2]:
import numpy as np
import pandas as pd
import gc
DATA_PATH = '../data/'

In [ ]:
print('loading prior')
priors = pd.read_csv(DATA_PATH + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv(DATA_PATH + 'order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading orders')
orders = pd.read_csv(DATA_PATH + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

loading prior
loading train


In [2]:
priors = priors.groupby('order_id').apply(lambda x: pd.Series(x.product_id.values)).unstack().to_sparse(fill_value=0)

In [31]:
train = train.groupby('order_id').apply(lambda x: pd.Series(x.product_id.values)).unstack().to_sparse(fill_value=0)

In [40]:
df = pd.concat([priors,train], axis=0)

In [53]:
df.to_pickle('full_unpackedprodid')

In [54]:
del priors, train
gc.collect;

In [71]:
print('loading orders')
orders = pd.read_csv(DATA_PATH + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

loading orders


In [72]:
train_orders = orders[orders.eval_set!='test']
test_orders = orders[orders.eval_set=='test']
df = df.set_index("order_id")

In [73]:
del orders
gc.collect();

In [74]:
train_final = train_orders.join(df, on="order_id", how="left")

In [3]:
#test_orders.to_pickle(DATA_PATH+"test.pickle")
#train_final.to_pickle(DATA_PATH+"train.pickle")
train_final = pd.read_pickle(DATA_PATH+'train.pickle')

In [4]:
train_final.reset_index(inplace=True)

In [5]:
total_per_user = train_final['order_number'].max()

In [6]:
train_final = train_final.join(train_final.groupby(['user_id'])['order_number'].max(),on='user_id', rsuffix='_max')

In [7]:
train_final['order_number_margin'] = total_per_user-train_final['order_number_max']

In [8]:
train_final['new_order_number'] = train_final['order_number'] + train_final['order_number_margin']

In [9]:
train_final.set_index(['user_id','new_order_number'], inplace=True)

In [10]:
train_final.drop(['index','order_number_margin','order_number_max'],axis=1,inplace=True)

In [16]:
user_id_indices = list(set([index[0] for index in train_final.index]))

In [19]:
new_multiindex = [[x,y] for x in user_id_indices for y in range(1,100+1)]

In [22]:
# Takes forever
train_final = train_final.reindex(new_multiindex)

In [23]:
train_final.to_pickle(DATA_PATH+'padded_train.pickle')

MemoryError: 

In [ ]:
train_final.memory_usage.sum()